# 1- Coded Page Rank Algorithm

## Core Page Rank algorithm

In [1]:
# import necessary libs
import numpy as np
import xml.etree.ElementTree as ET

In [2]:
def page_rank_score(graph, λ=0.85, ε=1.0e-6):
    N = len(graph)
    P = np.zeros((N, N))

    # Build the transition probability matrix P
    for i in range(N):
        out_links = sum(graph[i])
        if out_links == 0:
            P[i] = np.ones(N) / N  # Handle dangling nodes
        else:
            P[i] = graph[i] / out_links

    # Initialize PageRank values
    R = np.ones(N) / N
    delta = 1  # Difference between old and new ranks
    iterations = 0  # Counter to track the number of iterations

    # Iterate until the change is smaller than epsilon (ε)
    while delta > ε:
        R_new = λ * np.dot(R, P) + (1 - λ) / N  # Update PageRank values
        delta = np.linalg.norm(R_new - R, 1)  # Calculate the difference
        R = R_new
        iterations += 1  # Increment the iteration counter

    print(f"Converged in {iterations} iterations.")  # Show iteration count
    return R

In [3]:
# Print PageRank scores from any source
def print_page_rank(page_ranks, node_map):
    for node, index in node_map.items():
        percentage = page_ranks[index] * 100
        print(f"The PageRank of node {node}: {percentage:.1f}%")

### Method 1 : PageRank from adjacency matrix

In [4]:
def page_rank_score_method1(adj_matrix, λ=0.85, ε=1.0e-6):
    return page_rank_score(adj_matrix, λ, ε)

In [5]:
# Example: Adjacency matrix input
adj_matrix = np.array([
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],  # A
    [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],  # B
    [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],  # C
    [1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],  # D
    [0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0],  # E
    [0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0],  # F
    [0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0],  # G
    [0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0],  # H
    [0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0],  # I
    [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],  # L
    [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]   # M
])
node_ids = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'L', 'M']

In [6]:
# Call the functions to print PageRank scores
print("PageRank from Matrix:")
page_ranks = page_rank_score_method1(adj_matrix)
print_page_rank(page_ranks, {node: i for i, node in enumerate(node_ids)})

PageRank from Matrix:
Converged in 81 iterations.
The PageRank of node A: 3.3%
The PageRank of node B: 38.4%
The PageRank of node C: 34.3%
The PageRank of node D: 3.9%
The PageRank of node E: 8.1%
The PageRank of node F: 3.9%
The PageRank of node G: 1.6%
The PageRank of node H: 1.6%
The PageRank of node I: 1.6%
The PageRank of node L: 1.6%
The PageRank of node M: 1.6%


### Method 2 : PageRank from an XML file

In [7]:
# PageRank from XML graph representation
def page_rank_score_method2(xml_file, λ=0.85, ε=1.0e-6):
    tree = ET.parse(xml_file)
    root = tree.getroot()
    nodes = root.findall('.//node')

    # Map node IDs to indices
    node_id_map = {node.get('id'): index for index, node in enumerate(nodes)}
    N = len(nodes)
    graph = np.zeros((N, N))

    # Populate the graph matrix based on XML structure
    for node in nodes:
        node_id = node.get('id')
        for edge in node.findall('.//edge'):
            target_id = edge.get('target')
            if target_id in node_id_map:
                graph[node_id_map[node_id], node_id_map[target_id]] = 1

    return page_rank_score(graph, λ, ε), node_id_map

In [8]:
print("\nPageRank from XML:")
xml_file_path = 'C:/Users/dell/Downloads/BDSaS S3/graph-2.xml'
page_ranks, node_map = page_rank_score_method2(xml_file_path)
print_page_rank(page_ranks, node_map)


PageRank from XML:
Converged in 81 iterations.
The PageRank of node A: 3.3%
The PageRank of node B: 38.4%
The PageRank of node C: 34.3%
The PageRank of node D: 3.9%
The PageRank of node E: 8.1%
The PageRank of node F: 3.9%
The PageRank of node G: 1.6%
The PageRank of node H: 1.6%
The PageRank of node I: 1.6%
The PageRank of node L: 1.6%
The PageRank of node M: 1.6%


### Method 3 : PageRank from a graph

In [9]:
# PageRank from a manually defined graph
def page_rank_score_method3(nodes, edges, λ=0.85, ε=1.0e-6):
    N = len(nodes)  # Number of nodes
    graph = np.zeros((N, N))

    # Create a mapping of node names to indices
    node_index = {node: idx for idx, node in enumerate(nodes)}

    # Populate the adjacency matrix based on edges
    for src, dst in edges:
        graph[node_index[src], node_index[dst]] = 1

    return page_rank_score(graph, λ, ε), node_index

In [10]:
# Example: Graph input
nodes = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'L', 'M']
edges = [
    ('B', 'C'),  # B -> C
    ('C', 'B'),  # C -> B
    ('D', 'B'),  # D -> B
    ('D', 'A'),  # D -> A
    ('E', 'F'),  # E -> F
    ('E', 'B'),  # E -> B
    ('E', 'D'),  # E -> D
    ('F', 'B'),  # F -> B
    ('F', 'E'),  # F -> E
    ('G', 'E'),  # G -> E
    ('G', 'B'),  # G -> B
    ('H', 'B'),  # H -> B
    ('H', 'E'),  # H -> E
    ('I', 'B'),  # I -> B
    ('I', 'E'),  # I -> E
    ('L', 'E'),  # L -> E
    ('M', 'E')   # M -> E
]

In [11]:
print("\nPageRank from Graph:")
page_ranks, node_map = page_rank_score_method3(nodes, edges)
print_page_rank(page_ranks, node_map)


PageRank from Graph:
Converged in 81 iterations.
The PageRank of node A: 3.3%
The PageRank of node B: 38.4%
The PageRank of node C: 34.3%
The PageRank of node D: 3.9%
The PageRank of node E: 8.1%
The PageRank of node F: 3.9%
The PageRank of node G: 1.6%
The PageRank of node H: 1.6%
The PageRank of node I: 1.6%
The PageRank of node L: 1.6%
The PageRank of node M: 1.6%


# 2- Built-in PageRank function 

### with a Graph

In [24]:
import networkx as nx

# Example graph represented as a dictionary
graph = {
    'A': [],
    'B': ['C'],
    'C': ['B'],
    'D': ['B', 'A'],
    'E': ['F', 'B', 'D'],
    'F': ['B', 'E'],
    'G': ['E', 'B'],
    'H': ['B', 'E'],
    'I': ['B', 'E'],
    'L': ['E'],
    'M': ['E']
}

# Create a directed graph from the dictionary
G = nx.DiGraph()

# Add edges to the graph based on the input dictionary
for node, edges in graph.items():
    for edge in edges:
        G.add_edge(node, edge)

# Calculate PageRank using NetworkX directly
page_rank_scores_G = nx.pagerank(G, alpha=0.85)

# Output the PageRank scores formatted to one decimal place
for node, score in page_rank_scores_G.items():
    print(f"The PageRank of node {node}: {round(score * 100, 1)}%")

The PageRank of node B: 38.4%
The PageRank of node C: 34.3%
The PageRank of node D: 3.9%
The PageRank of node A: 3.3%
The PageRank of node E: 8.1%
The PageRank of node F: 3.9%
The PageRank of node G: 1.6%
The PageRank of node H: 1.6%
The PageRank of node I: 1.6%
The PageRank of node L: 1.6%
The PageRank of node M: 1.6%


### with an XML file

In [25]:
import xml.etree.ElementTree as ET
import networkx as nx

# Parse the XML file
tree = ET.parse('C:/Users/dell/Downloads/BDSaS S3/graph.xml')
root = tree.getroot()

# Create a directed graph
G = nx.DiGraph()

# Extract nodes and their outlinks from the XML
for node in root.findall('node'):
    node_id = node.get('id')
    for outlink in node.findall('outlink'):
        linked_node_id = outlink.text.strip()
        G.add_edge(node_id, linked_node_id)

# Calculate PageRank using NetworkX
page_rank_scores_X = nx.pagerank(G, alpha=0.85)

# Output the PageRank scores formatted to one decimal place
for node, score in page_rank_scores_X.items():
    print(f"The PageRank of node {node}: {round(score * 100, 1)}%")


The PageRank of node B: 38.4%
The PageRank of node C: 34.3%
The PageRank of node D: 3.9%
The PageRank of node A: 3.3%
The PageRank of node E: 8.1%
The PageRank of node F: 3.9%
The PageRank of node G: 1.6%
The PageRank of node H: 1.6%
The PageRank of node I: 1.6%
The PageRank of node L: 1.6%
The PageRank of node M: 1.6%


### with an adjancency matrix

In [28]:
import numpy as np
import networkx as nx

# Define the adjacency matrix
adj_matrix = np.array([
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],  # A
    [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],  # B
    [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],  # C
    [1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],  # D
    [0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0],  # E
    [0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0],  # F
    [0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0],  # G
    [0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0],  # H
    [0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0],  # I
    [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],  # L
    [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]   # M
])

# Create a directed graph from the adjacency matrix
G = nx.from_numpy_array(adj_matrix, create_using=nx.DiGraph)

# Define node labels (to match your original labels)
node_labels = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'L', 'M']

# Relabel the nodes in the graph
G = nx.relabel_nodes(G, {i: label for i, label in enumerate(node_labels)})

# Calculate PageRank using NetworkX
page_rank_scores_A = nx.pagerank(G, alpha=0.85)

for node, score in page_rank_scores_A.items():
    print(f"The PageRank of node {node}: {round(score * 100, 1)}%")


The PageRank of node A: 3.3%
The PageRank of node B: 38.4%
The PageRank of node C: 34.3%
The PageRank of node D: 3.9%
The PageRank of node E: 8.1%
The PageRank of node F: 3.9%
The PageRank of node G: 1.6%
The PageRank of node H: 1.6%
The PageRank of node I: 1.6%
The PageRank of node L: 1.6%
The PageRank of node M: 1.6%
